In [ ]:
import cv2
import dlib
import math
import unittest
import numpy as np
import urllib.request

from scipy.spatial import distance
from matplotlib import pyplot as plt
!wget -O ./shape_predictor_68_face_landmarks.dat "https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%206%20-%2010%20(Projects)/Project%20-%20Emotion%20Detection/shape_predictor_68_face_landmarks.dat"
dlibshape_path ='./shape_predictor_68_face_landmarks.dat'

In [ ]:
frontalface_detector = dlib.get_frontal_face_detector()

In [ ]:
def rect_to_bb(rect):
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    return (x, y, w, h)

def detect_face(image_url):
  try:
    url_response = urllib.request.urlopen(image_url)
    img_array = np.array(bytearray(url_response.read()), dtype=np.uint8)
    image = cv2.imdecode(img_array, -1)
  except Exception as e:
    return "Please check the URL and try again!"
    
  rects = frontalface_detector(image, 1)
  
  if len(rects) < 1:
    return "No Face Detected"
  
  for (i, rect) in enumerate(rects):
    (x, y, w, h) = rect_to_bb(rect)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
  plt.imshow(image, interpolation='nearest')
  plt.axis('off')
  plt.show()

In [ ]:
detect_face(input('Enter the URL of the image: '));  # run cell and when prompted, input a URL of an img and press 'enter'!

In [ ]:
frontalface_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('./shape_predictor_68_face_landmarks.dat')

In [ ]:
def get_landmarks(image_url):
  try:
    url_response = urllib.request.urlopen(image_url)
    img_array = np.array(bytearray(url_response.read()), dtype=np.uint8)
    image = cv2.imdecode(img_array, -1)
    
  except Exception as e:
    print ("Please check the URL and try again!")
    return None,None
  
  faces = frontalface_detector(image, 1)
  if len(faces):
    landmarks = [(p.x, p.y) for p in landmark_predictor(image, faces[0]).parts()]
  else:
    return None,None
  
  return image,landmarks

In [ ]:
def plot_image_landmarks(image,face_landmarks):
  radius = -1
  circle_thickness = 5
  image_copy = image.copy()
  for (x, y) in face_landmarks:
    cv2.circle(image_copy, (x, y), circle_thickness, (255,0,0), radius)
  plt.imshow(image_copy, interpolation='nearest')
  plt.axis('off')
  plt.show()

In [ ]:
image,landmarks= get_landmarks(input("Enter the URL of the image: ")) #url
if landmarks:
  plot_image_landmarks(image,landmarks)
else:
  print ("No Landmarks Detected")

In [ ]:
def show_indices(landmarks, i_index): 
  
  plt.scatter(x=[landmarks[i][0] for i in range(len(landmarks)//2, len(landmarks))], 
              y=[-landmarks[i][1] for i in range(len(landmarks)//2, len(landmarks))], s=50, alpha=.5, color='blue', label='second half of indices') 

  plt.scatter(x=[landmarks[i][0] for i in range(len(landmarks)//2)], 
              y=[-landmarks[i][1] for i in range(len(landmarks)//2)], color='red', alpha=.5, label='first half of indices')

  x = landmarks[i_index][0]
  y = -landmarks[i_index][1]
  plt.scatter(x=x, y=y, 
             color='purple', s=100, marker='x', label='feature at index %d'%i_index)
  
  plt.scatter(x, y, color='red', alpha=.5, label='selected indices')

  plt.axis('off');
  plt.legend(bbox_to_anchor=[1,1]);
  plt.title('Visualizing the features we\'ve extracted from this image',y =1.2); 

In [ ]:
show_index = 30
show_indices(landmarks, show_index)
np.array(landmarks).shape

In [ ]:
landmark_indices = {'eyes':(36,47),
                    "nose":(27,35),
                    "mouth":(48,67),
                    "jawline":(0,17),
                    "eyebrow":(18,27)}


print(landmark_indices["eyes"])

In [ ]:
eye_points = np.array([36,47])
selected_landmarks = landmarks[eye_points[0]:eye_points[1]+1]
print(selected_landmarks)
plot_image_landmarks(image,selected_landmarks)

In [ ]:
FACIAL_LANDMARKS_IDXS = {"EYES":(36,47),
                         "NOSE":(27,35),
                         "MOUTH":(48,67),
                        "JAWLINE":(0,17),
                        "EYEBROWS":(18,27)}

for key,value in FACIAL_LANDMARKS_IDXS.items():
  print (key,"DETECTION")
  selected_landmarks = landmarks[value[0]:value[1]+1]
  plot_image_landmarks(image, selected_landmarks)

In [ ]:
def euclidean_distance(p1,p2):
  distance =  math.sqrt((p2[0]-p1[0])**2 + (p2[1]-p1[1])**2)
  return distance

In [ ]:
def classify_images(image1_path, image2_path, plt_flag):
  
  image1,image1_landmarks = get_landmarks(image1_path)
  image2,image2_landmarks = get_landmarks(image2_path)
  
  if plt_flag:
    plt.imshow(image1, interpolation='nearest')
    plt.title("Image1")
    plt.show()

    plt.imshow(image2, interpolation='nearest')
    plt.title("Image2")
    plt.show()
    
  
  pairs_distance = [(37,41),(38,40),(43,47),(44,46)]
  
  e_sum1 = 0
  e_sum2 = 0
  threshold_value = 10
  for pair in pairs_distance:
    
    e_sum1 = e_sum1 + euclidean_distance(image1_landmarks[pair[0]],
                                         image1_landmarks[pair[1]])
    e_sum2 = e_sum2 + euclidean_distance(image2_landmarks[pair[0]],
                                         image2_landmarks[pair[1]])
  print (e_sum1,e_sum2)
  
  e_difference = e_sum1 - e_sum2
  print (e_difference)
  if int(e_difference) == 0:
    return ("Both images have eyes open or closed")
  
  if abs(e_difference) >= threshold_value:
     
    if e_difference > 0:
        return ("Image1 : Eyes Open, Image2 : Eyes Close")
    else:
        return ("Image1 : Eyes Close, Image2 : Eyes Open")